In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical

# load data

In [ ]:
data=pd.read_csv("../input/hand-gesture-recognition-2nd-edition/train.csv")

In [ ]:
data.head()

In [ ]:
plt.imshow(np.array(data.drop(['id','label'],axis=1).iloc[5]).reshape(211,271))

# split data

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(data.drop(['id','label'],axis=1),data['label'],test_size=0.2,random_state=42)

# data processing

In [ ]:
y_train=to_categorical(y_train)

In [ ]:
y_test=to_categorical(y_test)

In [ ]:
X_train =X_train.values.reshape(-1,211,271,1)
X_test = X_test.values.reshape(-1,211,271,1)

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(X_train)



# model cnn

In [ ]:
model=keras.Sequential()
model.add(layers.Conv2D(128,(5,5),padding='same',activation='relu',input_shape=X_train.shape[1:]))
model.add(layers.Dropout(0.3))
model.add(layers.Conv2D(64,(5,5),padding='same',activation='relu'))
model.add(layers.Conv2D(32,(5,5),padding='same',activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Flatten())
model.add(layers.Dense(17,activation='softmax'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
epochs=10
batch_size=10
History =model.fit(X_train,y_train, batch_size=batch_size,
                              epochs = epochs, validation_data = (X_test,y_test),
                              )

In [ ]:
from sklearn.metrics import accuracy_score

# prediction

In [ ]:
results = model.predict(X_test)
pred=np.argmax(results,axis=1)
y=np.argmax(y_test,axis=1)
accuracy_score(pred,y)

# Submission

In [ ]:
dfa=pd.read_csv('../input/hand-gesture-recognition-2nd-edition/test.csv')

In [ ]:
dfa.head()

In [ ]:
X=dfa.drop('id',axis=1).values.reshape(-1,211,271,1)

In [ ]:
results = model.predict(X)
y=np.argmax(results,axis=1)


In [ ]:
sub=pd.read_csv('../input/hand-gesture-recognition-2nd-edition/sample_submission.csv')

In [ ]:
sub['label']=np.array(y)

In [ ]:
sub.to_csv("sub.csv",index=False)